# Gaussian Distributions Utils
> Functions to help work with gaussian distributions

In [ ]:
#| hide
#| default_exp gaussian

In [ ]:
from fastcore.test import *

## Normal Parameters

In [ ]:
#| export
from collections import namedtuple
from fastcore.basics import patch

### Normal

In [ ]:
import torch

In [ ]:
#| export
ListNormal = namedtuple('ListNormal', ['mean', 'std'])

In [ ]:
#| export
Normal = namedtuple('Normal', ['mean', 'std'])

In [ ]:
#| export
@patch
def __getitem__(self: ListNormal, n:int
           )->Normal:
    """Get the mean and cov for the nth Normal distribution in the list """
    return Normal(self.mean[n], self.std[n])

In [ ]:
#| export
@patch
def detach(self: ListNormal)->ListNormal:
    """Detach both mean and cov at once """
    return ListNormal(self.mean.detach(), self.std.detach())

In [ ]:
ListNormal(torch.rand(10), torch.rand(10))[1]

Normal(mean=tensor(0.8288), std=tensor(0.8127))

### Multivariate Normal

In [ ]:
#| export
ListMNormal = namedtuple('ListMultiNormal', ['mean', 'cov'])

In [ ]:
#| export
MNormal = namedtuple('MultiNormal', ['mean', 'cov'])

In [ ]:
#| export
@patch
def __getitem__(self: ListMNormal, n:int
           )->Normal:
    """Get the mean and cov for the nth Normal distribution in the list """
    return MNormal(self.mean[n], self.cov[n])
@patch
def __setitem__(self: ListMNormal, idx, value)->Normal:
    """set the mean and cov for the nth Normal distribution in the list """
    self.mean[idx], self.cov[idx] = value

In [ ]:
#| export
@patch
def detach(self: ListMNormal)->ListMNormal:
    """Detach both mean and cov at once """
    return ListMNormal(self.mean.detach(), self.cov.detach())

In [ ]:
ListMNormal(torch.rand(2,10), torch.rand(2,10,10))[1]

MultiNormal(mean=tensor([0.5525, 0.8355, 0.6809, 0.4615, 0.2613, 0.2542, 0.8745, 0.1663, 0.7594,
        0.9829]), cov=tensor([[0.8970, 0.3792, 0.4856, 0.4547, 0.5250, 0.3540, 0.6135, 0.1249, 0.9350,
         0.5199],
        [0.6439, 0.2499, 0.2646, 0.5759, 0.0825, 0.7846, 0.7053, 0.7155, 0.5080,
         0.9545],
        [0.7253, 0.7372, 0.0291, 0.1926, 0.9085, 0.6373, 0.6138, 0.5850, 0.1298,
         0.7790],
        [0.6195, 0.4928, 0.2752, 0.9113, 0.6431, 0.9983, 0.4997, 0.5400, 0.4034,
         0.1039],
        [0.7134, 0.6939, 0.4348, 0.9082, 0.3838, 0.3346, 0.2036, 0.7074, 0.7747,
         0.5095],
        [0.5271, 0.4253, 0.1755, 0.1011, 0.1805, 0.0845, 0.8968, 0.7160, 0.9273,
         0.8917],
        [0.1423, 0.3399, 0.1476, 0.3554, 0.9244, 0.9693, 0.1446, 0.3901, 0.2852,
         0.2318],
        [0.0128, 0.5210, 0.3184, 0.9317, 0.0466, 0.9699, 0.3329, 0.4150, 0.4741,
         0.6318],
        [0.3606, 0.7720, 0.4114, 0.5597, 0.1642, 0.2645, 0.0797, 0.3353, 0.9760,
        

## Positive Definite

The covariance matrices need to be [positive definite](https://en.wikipedia.org/wiki/Definite_matrix)
Those are utilities functions to check is a matrix is positive definite and to make any matrix positive definite

In [ ]:
#| export
import pandas as pd
from torch import Tensor

In [ ]:
A = torch.rand(3,3) # random matrix used for testing

#### Symmetry

In [ ]:
#| export
def is_symmetric(value, atol=1e-5):
    return torch.isclose(value, value.mT, atol=atol).all().item()

In [ ]:
is_symmetric(A)

False

In [ ]:
#| export
def symmetric_upto(value, start=-8):
    for exp in torch.arange(start, 3):
        if is_symmetric(value, atol=10**exp):
            return exp.item()
    return exp.item()

In [ ]:
symmetric_upto(A)

0

#### is posdef

Default pytorch check (uses symmetry + cholesky decomposition)

In [ ]:
#| export
def is_posdef(cov):
    return torch.distributions.constraints.positive_definite.check(cov).item()

In [ ]:
is_posdef(A)

False

check if it is pos definite using eigenvalues. Positive definite matrix have all positive eigenvalues

In [ ]:
torch.linalg.eigvalsh(A)

tensor([0.1237, 0.4340, 1.1856])

In [ ]:
#| export
def is_posdef_eigv(cov):
    eigv = torch.linalg.eigvalsh(cov)
    if (eigv < 0).any():
        return False, eigv
    return True, eigv

In [ ]:
is_posdef_eigv(A)

(True, tensor([0.1237, 0.4340, 1.1856]))

### Pytorch constraint

Note that `is_posdef` and `is_posdef_eigv` can return different values, in general `is_posdef_eigv` is more tollerant

transform any matrix $A$ into a positive definite matrix ($PD$) using the following formula

$PD = CC^T$ where $C$ is the lower triangular matrix of $A$

the inverse transformation uses cholesky decomposition


The API inspired by gpytorch constraints

In [ ]:
#| export
class PosDef():
    """ Positive Definite Constraint for PyTorch parameters"""
    def transform(self,
                  raw # square matrix
                 ):
        """transform any matrix into a positive definite one"""
        C = torch.tril(raw)
        return C @ C.mT
    
    def inverse_transform(self,
                          value # a positive definite matrix
                         ):
        """tranform positive definite matrix into a matrix that can be back_transformed using `transform`"""
        return torch.linalg.cholesky(value)

to_posdef = PosDef().transform

In [ ]:
constraint = PosDef()

posdef = constraint.transform(A)

In [ ]:
A

tensor([[0.6099, 0.4402, 0.0249],
        [0.4237, 0.6002, 0.7070],
        [0.3168, 0.1285, 0.5331]])

In [ ]:
posdef

tensor([[0.3720, 0.2584, 0.1932],
        [0.2584, 0.5398, 0.2113],
        [0.1932, 0.2113, 0.4011]])

In [ ]:
test_eq(is_posdef(posdef), True)

In [ ]:
constraint.inverse_transform(posdef)

tensor([[0.6099, 0.0000, 0.0000],
        [0.4237, 0.6002, 0.0000],
        [0.3168, 0.1285, 0.5331]])

In [ ]:
test_close(posdef, constraint.transform(constraint.inverse_transform(posdef)))

In [ ]:
is_symmetric(posdef)

True

In [ ]:
symmetric_upto(posdef)

-8

In [ ]:
is_posdef_eigv(posdef)

(True, tensor([0.1719, 0.2482, 0.8927]))

#### Check pos def

This is to help finding matrices that aren't positive definite and debug the issues.
Returns a detailed dataframe row with info about the matrix and optionally logs everything to a global object

In [ ]:
#| export
from warnings import warn
from fastcore.basics import store_attr

In [ ]:
#| export
class CheckPosDef():
    def __init__(self,
                do_check:bool = False, # set to True to actually check matrix
                use_log:bool = True, # keep internal log
                warning:bool = True, # show a warning if a matrix is not pos def 
                ):
        store_attr()
        self.log = pd.DataFrame()
        self.extra_args = {}
    def add_args(self, **kwargs):
        """Add an extra argument to the next call of check_posdef """
        self.extra_args = {**kwargs, **self.extra_args}
        return self
    
    def check(self,
              x: Tensor, # (batch of) square matrix
              **extra_args
             ) -> pd.DataFrame:
        
        if not self.do_check: return
        
        self.add_args(**extra_args)
        
        x = x if x.dim() > 2 else [x]
        infos = pd.concat([*map(self._check_matrix, x)])
        
        if self.use_log: self.log = pd.concat([self.log, infos])
        if self.warning and (~infos['is_pd_eigv'].all() or ~infos['is_pd_chol'].all()):
             warn("Matrix is not positive definite")
        
        self.extra_args = {} 
        return infos
    
    def _check_matrix(self,
                     x: Tensor # square matrix
                    ) -> pd.DataFrame:
        
        x = x.detach().cpu().clone() # free GPU memory and ensure that there is a copy
        sym_upto = symmetric_upto(x)

        is_pd_eigv, eigv = is_posdef_eigv(x)
        is_pd_chol = torch.linalg.cholesky_ex(x).info.eq(0).all().item() # skip pytorch too strict symmetry check
        is_sym = is_symmetric(x)

        info = pd.DataFrame({
            'is_pd_eigv': is_pd_eigv,
            'is_pd_chol': is_pd_chol,
            'is_sym': is_sym,
            'sym_upto': sym_upto,
            'eigv': [eigv.detach().numpy()],
            'matrix': [x.detach().numpy()],
            **self.extra_args
        })

        return info

In [ ]:
CheckPosDef(True).check(A)

,is_pd_eigv,is_pd_chol,is_sym,sym_upto,eigv,matrix
0,True,True,False,0,"[0.123707384, 0.43397072, 1.1855843]","[[0.6098797, 0.4402153, 0.024890661], [0.42369..."


In [ ]:
checker = CheckPosDef(True)

checker.check(A, my_arg="my arg") # this will be another col in the log

,is_pd_eigv,is_pd_chol,is_sym,sym_upto,eigv,matrix,my_arg
0,True,True,False,0,"[0.123707384, 0.43397072, 1.1855843]","[[0.6098797, 0.4402153, 0.024890661], [0.42369...",my arg


In [ ]:
checker.log

,is_pd_eigv,is_pd_chol,is_sym,sym_upto,eigv,matrix,my_arg
0,True,True,False,0,"[0.123707384, 0.43397072, 1.1855843]","[[0.6098797, 0.4402153, 0.024890661], [0.42369...",my arg


In [ ]:
checker.add_args(show="only once")
checker.check(posdef)
checker.check(A)
checker.log

,is_pd_eigv,is_pd_chol,is_sym,sym_upto,eigv,matrix,my_arg,show
0,True,True,False,0,"[0.123707384, 0.43397072, 1.1855843]","[[0.6098797, 0.4402153, 0.024890661], [0.42369...",my arg,NaN
0,True,True,True,-8,"[0.17188226, 0.24823612, 0.89274585]","[[0.37195322, 0.25839996, 0.19320545], [0.2583...",NaN,only once
0,True,True,False,0,"[0.123707384, 0.43397072, 1.1855843]","[[0.6098797, 0.4402153, 0.024890661], [0.42369...",NaN,NaN


In [ ]:
B = torch.rand(2,3,3) # a batch of matrices

In [ ]:
checker.check(B)

/tmp/ipykernel_38128/4164403042.py:30: UserWarning: Matrix is not positive definite
  warn("Matrix is not positive definite")


,is_pd_eigv,is_pd_chol,is_sym,sym_upto,eigv,matrix
0,False,False,False,0,"[-0.029349297, 0.76472014, 1.1787026]","[[0.15079308, 0.33835346, 0.36433917], [0.3684..."
0,False,False,False,0,"[-0.2295478, 0.36021793, 0.8472092]","[[0.47898299, 0.31957358, 0.80752444], [0.0928..."


In [ ]:
test_close(B[0] @ A, (B @ A)[0]) # example batched matrix multiplication

## Conditional Predictions

Therefore we need to compute the conditional distribution of a normal ^[https://cs.nyu.edu/~roweis/notes/gaussid.pdf eq, 5a, 5d]

$$ X = \left[\begin{array}{c} x \\ o \end{array} \right] $$

$$ p(X) = N\left(\left[ \begin{array}{c} \mu_x \\ \mu_o \end{array} \right], \left[\begin{array}{cc} \Sigma_{xx} & \Sigma_{xo} \\ \Sigma_{ox} & \Sigma_{oo} \end{array} \right]\right)$$

where $x$ is a vector of variable that need to predicted and $o$ is a vector of the variables that have been observed


then the conditional distribution is:

$$p(x|o) = N(\mu_x + \Sigma_{xo}\Sigma_{oo}^{-1}(o - \mu_o), \Sigma_{xx} - \Sigma_{xo}\Sigma_{oo}^{-1}\Sigma_{ox})$$

In [ ]:
#| export
import torch
from torch.distributions import MultivariateNormal
from torch.linalg import cholesky
from torch import cholesky_inverse
from torch import Tensor

from fastcore.test import *
from meteo_imp.utils import *
from typing import List

In [ ]:
#| export
def conditional_guassian(
                         μ: Tensor, # mean with shape `[n_vars]`
                         Σ: Tensor, # cov with shape `[n_vars, n_vars] `
                         obs: Tensor, # Observations with shape `[n_obs]`, where `n_obs = sum(idx)`
                         mask: Tensor # Boolean tensor specifying for each variable is observed (True) or not (False). Shape `[n_vars]`
                        ) -> ListMNormal: # Distribution conditioned on observations. shape `[n_vars - n_obs]`
    assert μ.shape[0] == mask.shape[0]
    assert obs.shape[0] == sum(mask)
    
    μ_x = μ[~mask]
    μ_o = μ[mask]
    # the double square brackets `:][:` are needed to keep the dimensionality even for empty tensors 
    Σ_xx = Σ[~mask,:][:, ~mask]
    Σ_xo = Σ[~mask,:][:,  mask]
    Σ_ox = Σ[ mask,:][:, ~mask]
    Σ_oo = Σ[ mask,:][:,  mask]
    
    Σ_oo_inv = cholesky_inverse(cholesky(Σ_oo))
    
    
    mean = μ_x + Σ_xo@Σ_oo_inv@(obs - μ_o)
    cov = Σ_xx - Σ_xo@Σ_oo_inv@Σ_ox
    
    return ListMNormal(mean, cov)
    

In [ ]:
# example distribution with only 2 variables
μ = torch.tensor([.5, 1.])
Σ = torch.tensor([[1., .5], [.5 ,1.]])


mask = torch.tensor([True, False]) # second variable is the observed one

obs = torch.tensor([5.]) # value of second variable

gauss_cond = conditional_guassian(μ, Σ, obs, mask)

# hardcoded values to test that the code is working, see also for alternative implementation https://python.quantecon.org/multivariate_normal.html
test_close(3.25, gauss_cond.mean.item())
test_close(.75, gauss_cond.cov.item())

### Batches

cannot have proper batch support, or at least not in a straigthforward way as the shape of the output would be different for the different batches.

so using a for-loop to temporarly fix the situation

In [ ]:
#| export
def cond_gaussian_batched(dist: ListMNormal,
                         obs, # this needs to have the same shape of the mask !!! 
                         mask
                         ) -> List[ListMNormal]: # lists of distributions for element in the batch
    return [conditional_guassian(dist.mean[i], dist.cov[i], obs[i][mask[i]], mask[i]) for i in range(obs.shape[0])]
        

In [ ]:
reset_seed(10)
mean = torch.rand(2,3) # batch
cov = to_posdef(torch.rand(2,3,3))
mask = torch.rand(2,3) > .3
obs = torch.rand(2,3)

In [ ]:
conditional_gaussian_batched(mean, cov, obs, mask)

[ListMultiNormal(mean=tensor([0.4581, 0.4829, 0.3125]), cov=tensor([[0.4814, 0.2292, 0.4885],
         [0.2292, 0.4094, 0.5379],
         [0.4885, 0.5379, 1.2905]])),
 ListMultiNormal(mean=tensor([-0.3978, -1.0285]), cov=tensor([[0.6654, 0.2320],
         [0.2320, 0.1665]]))]

In [ ]:
mask.shape, obs.shape

(torch.Size([2, 3]), torch.Size([2, 3]))

In [ ]:
assert mean.shape == mask.shape
assert mean.dim() == 2

In [ ]:
obs.shape

torch.Size([2, 3])

In [ ]:
mean_x = mean[~mask]
mean_o = mean[mask]

In [ ]:
mask

tensor([[False, False, False],
        [False, False,  True]])

In [ ]:
mean_x

tensor([0.4581, 0.4829, 0.3125, 0.6150, 0.2139])

In [ ]:
cov.shape

torch.Size([2, 3, 3])

In [ ]:
cov[~mask]

tensor([[0.4814, 0.2292, 0.4885],
        [0.2292, 0.4094, 0.5379],
        [0.4885, 0.5379, 1.2905],
        [0.9701, 0.6057, 0.0567],
        [0.6057, 0.6250, 0.0695]])

In [ ]:
cov

tensor([[[0.4814, 0.2292, 0.4885],
         [0.2292, 0.4094, 0.5379],
         [0.4885, 0.5379, 1.2905]],

        [[0.9701, 0.6057, 0.0567],
         [0.6057, 0.6250, 0.0695],
         [0.0567, 0.0695, 0.0105]]])

In [ ]:
cov[0][~mask[0], ~mask[0]]

tensor([0.4814, 0.4094, 1.2905])

In [ ]:
cov[0][mask[0],:][:, mask[0]].shape

torch.Size([0, 0])

### Performance

analysis of the performance of inverting a positive definite matrix

Use `cholesky` decomposition and `cholesky_solve` to improve performance of matrix inversion

see the [Probabilist machine learning course from uni Tübigen](https://uni-tuebingen.de/en/180804), specifically the code from the [Gaussian Regression Notebook](https://uni-tuebingen.de/fileadmin/Uni_Tuebingen/Fakultaeten/MatNat/Fachbereiche/Informatik/Lehrstuehle/MethMaschLern/Probabilistic_ML/Notebook_Vorlesung_7___9/Gaussian_Linear_Regression.ipynb) for details

This is the direct implementation of the equations

In [ ]:
def _conditional_guassian_base(
                         μ: Tensor, # mean with shape `[n_vars]`
                         Σ: Tensor, # cov with shape `[n_vars, n_vars] `
                         obs: Tensor, # Observations with shape `[n_vars]`
                         idx: Tensor # Boolean tensor specifying for each variable is observed (True) or not (False). Shape `[n_vars]`
                        ) -> ListNormal: # Distribution conditioned on observations
    μ_x = μ[~idx]
    μ_o = μ[idx]
    
    Σ_xx = Σ[~idx,:][:, ~idx]
    Σ_xo = Σ[~idx,:][:, idx]
    Σ_ox = Σ[idx,:][:, ~idx]
    Σ_oo = Σ[idx,:][:, idx]
    
    Σ_oo_inv = torch.linalg.inv(Σ_oo)
    
    mean = μ_x + Σ_xo@Σ_oo_inv@(obs - μ_o)
    cov = Σ_xx - Σ_xo@Σ_oo_inv@Σ_ox
    
    return ListNormal(mean, cov)
    

 faster version

In [ ]:
n_var = 5
mean = torch.rand(n_var, dtype=torch.float64)
cov = to_posdef(torch.rand(n_var, n_var, dtype=torch.float64))
dist = MultivariateNormal(mean, cov)
idx = torch.rand(n_var, dtype=torch.float64) > .5
obs = torch.rand(n_var, dtype=torch.float64)[idx]

In [ ]:
torch.linalg.inv(cov) 

tensor([[ 9.6648, -0.2920,  1.4796,  1.7772, -6.8714],
        [-0.2920,  1.6547, -0.3568, -0.7395,  0.5746],
        [ 1.4796, -0.3568,  5.0445,  0.1818, -5.9583],
        [ 1.7772, -0.7395,  0.1818,  1.4627, -1.5800],
        [-6.8714,  0.5746, -5.9583, -1.5800, 10.9719]], dtype=torch.float64)

In [ ]:
(torch.linalg.inv(cov) - cholesky_inverse(torch.linalg.cholesky(cov))).max()

tensor(7.9936e-15, dtype=torch.float64)

In [ ]:
test_close(torch.linalg.inv(cov), cholesky_inverse(torch.linalg.cholesky(cov)), eps=1e-2)

In [ ]:
reset_seed()
A = to_posdef(torch.rand(1000, 1000, dtype=torch.float64)) + torch.eye(1000) * 1e-3 # noise to ensure is positive definite

In [ ]:
is_symmetric(A)

True

In [ ]:
is_posdef(A)

True

In [ ]:
%timeit torch.linalg.inv(A)

16 ms ± 689 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
%timeit cholesky_inverse(torch.linalg.cholesky(A))

9.74 ms ± 143 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


The second version is way faster

In [ ]:
test_close(conditional_guassian(mean, cov, obs, idx).mean, _conditional_guassian_base(mean, cov, obs, idx).mean)

In [ ]:
B = to_posdef(torch.rand(n_var, n_var, dtype=torch.float64))

In [ ]:
B @ torch.inverse(cov)

tensor([[-3.7817e-01,  6.0855e-02, -3.5677e-01, -9.8335e-02,  6.4587e-01],
        [-1.2011e+00,  1.5680e-01, -1.1038e+00, -2.7579e-01,  2.0130e+00],
        [-2.6674e+00,  1.5877e-02, -9.0071e-01, -4.3940e-01,  2.9151e+00],
        [-7.7354e+00, -2.2823e-02, -7.3321e+00, -6.3090e-01,  1.2912e+01],
        [-1.6419e+01,  7.5170e-01, -1.4601e+01, -2.4740e+00,  2.6638e+01]],
       dtype=torch.float64)

In [ ]:
torch.cholesky_solve(cholesky(cov), B)

tensor([[ 1.3082e+04, -9.3404e+02,  2.4098e+00,  8.5058e-01, -1.5294e-02],
        [-6.2772e+02,  3.6820e+02, -2.3551e+00, -2.3026e-02, -9.6162e-02],
        [ 4.9489e+00, -1.5093e+00,  3.5311e+00, -3.0811e-01, -1.6152e-02],
        [ 2.5098e-01,  4.9503e-01,  6.9657e-02,  8.1607e-01, -4.3506e-02],
        [ 8.3257e-03, -3.8040e-01, -5.2151e-02, -1.1511e-01,  3.3907e-02]],
       dtype=torch.float64)

## Helper

### cov2std

In [ ]:
x = torch.stack([torch.eye(3)*i for i in  range(1,4)])

In [ ]:
x

tensor([[[1., 0., 0.],
         [0., 1., 0.],
         [0., 0., 1.]],

        [[2., 0., 0.],
         [0., 2., 0.],
         [0., 0., 2.]],

        [[3., 0., 0.],
         [0., 3., 0.],
         [0., 0., 3.]]])

In [ ]:
torch.diagonal(x, dim1=1, dim2=2)

tensor([[1., 1., 1.],
        [2., 2., 2.],
        [3., 3., 3.]])

In [ ]:
#| export
def cov2std(x):
    "convert cov of array of covariances to array of stddev"
    return torch.sqrt(torch.diagonal(x, dim1=-2, dim2=-1))

## Export

In [ ]:
#| hide
from nbdev import nbdev_export
nbdev_export()